## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from datetime import datetime
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dzhebariki-Khaya,RU,62.2167,135.8000,18.88,90,100,5.86,overcast clouds
1,1,Baghdad,IQ,33.3406,44.4009,84.20,26,0,3.44,clear sky
2,2,New Norfolk,AU,-42.7826,147.0587,54.00,76,100,1.01,overcast clouds
3,3,Sobolevo,RU,54.4299,31.8996,44.33,83,95,10.47,overcast clouds
4,4,Kapaa,US,22.0752,-159.3190,69.80,78,90,4.61,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Baghdad,IQ,33.3406,44.4009,84.20,26,0,3.44,clear sky
8,8,Rosetta,EG,31.4044,30.4164,86.00,37,0,6.91,clear sky
9,9,Dondo,MZ,-19.6094,34.7431,75.20,64,40,4.70,scattered clouds
15,15,Rikitea,PF,-23.1203,-134.9692,76.96,74,2,7.18,clear sky
16,16,Grand Gaube,MU,-20.0064,57.6608,77.00,85,100,3.00,light rain
17,17,Namibe,AO,-15.1961,12.1522,81.41,70,20,11.03,few clouds
19,19,Mahwa,IN,27.0471,76.9288,87.60,12,10,4.59,clear sky
20,20,Faanui,PF,-16.4833,-151.7500,82.02,72,9,11.61,clear sky
22,22,Camopi,GF,3.1655,-52.3158,80.85,81,84,4.00,broken clouds
23,23,Rawson,AR,-43.3002,-65.1023,75.85,33,67,15.77,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Baghdad,IQ,84.20,clear sky,33.3406,44.4009,
8,Rosetta,EG,86.00,clear sky,31.4044,30.4164,
9,Dondo,MZ,75.20,scattered clouds,-19.6094,34.7431,
15,Rikitea,PF,76.96,clear sky,-23.1203,-134.9692,
16,Grand Gaube,MU,77.00,light rain,-20.0064,57.6608,
17,Namibe,AO,81.41,few clouds,-15.1961,12.1522,
19,Mahwa,IN,87.60,clear sky,27.0471,76.9288,
20,Faanui,PF,82.02,clear sky,-16.4833,-151.7500,
22,Camopi,GF,80.85,broken clouds,3.1655,-52.3158,
23,Rawson,AR,75.85,broken clouds,-43.3002,-65.1023,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.       
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl><dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure


In [16]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

In [17]:
fig

Figure(layout=FigureLayout(height='420px'))